In [25]:
!pip install transformers
!pip install sentencepiece


In [13]:
#example to demonstrate how dataset is stored
with open('hi_train.conll',encoding='utf8') as f:
    j=1
    for i in f:
        j+=1
        if j==20:
            break
        print(i)

# id fb77da5d-c4e6-4545-ac07-600a7ff04920	domain=train

जियोर्जियोन _ _ O

०१९। _ _ O

जेपीजी _ _ O

| _ _ O

जियोर्जियोन _ _ O

आंधी _ _ B-CW

, _ _ O

८२ _ _ O

× _ _ O

७३ _ _ O

सेमी _ _ O



# id cdfbbb76-87a6-49d3-806e-9587d7c896f0	domain=train

इस _ _ O

क्षेत्र _ _ O

का _ _ O

सबसे _ _ O



In [14]:
#printing the above shows that there is a line followed by hash symbol. and there is a space before that line. All the words before
#that line belong to one sentence
# I = inside
# O = outside
# B= beginning of word



In [15]:
#we need to make a dataframe consisting of words in column and corresponding sentence number to which the word belongs to and
#also the corresponding tag of word
import re
sentence_id_list=list()
all_words_list=list()
tag_list=list()
sentence_id=0
with open('hi_train.conll',encoding='utf8') as f:
    for line in f:
        
        if line[0]=='#' or line[0]=='\n':
            if line[0]!='\n':
                sentence_id+=1
            continue
        else:
            
            element=line.split(' _ _ ')
            all_words_list.append(element[0])
            tag_list.append(re.sub('\n','',element[1]))
            sentence_id_list.append(sentence_id)
        

In [16]:
import pandas as pd
my_df=pd.DataFrame(columns=['sentence_id','word','tag'])
my_df['sentence_id']=sentence_id_list
my_df['word']=all_words_list
my_df['tag']=tag_list



In [17]:
tags_arr=my_df.tag.unique()
labels_mapping={}
for i in range(13):
    labels_mapping[tags_arr[i]]=i
    
labels_reverse_mapping={}
for i in range(13):
    labels_reverse_mapping[i]=tags_arr[i]
    
labels_reverse_mapping

{0: 'O',
 1: 'B-CW',
 2: 'B-PROD',
 3: 'B-PER',
 4: 'I-PER',
 5: 'I-PROD',
 6: 'B-LOC',
 7: 'B-CORP',
 8: 'I-CORP',
 9: 'B-GRP',
 10: 'I-GRP',
 11: 'I-CW',
 12: 'I-LOC'}

In [18]:
my_df.tag.unique()[0]

'O'

In [19]:
#grouping together words of same sentence
my_df['sentence']=my_df.groupby(['sentence_id'])['word'].transform(lambda func: ' '.join(func))
my_df['word labels']=my_df.groupby(['sentence_id'])['tag'].transform(lambda func: ','.join(func))

In [20]:
my_df = my_df.drop_duplicates(subset=['sentence_id'])

In [21]:
my_df=my_df[['sentence_id','sentence','word labels']]


In [22]:
len(all_words_list),len(sentence_id_list),len(tag_list)

(244439, 244439, 244439)

In [28]:
python -v

NameError: name 'python' is not defined

In [26]:
import transformers

MAX_LEN = 20
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 6*1e-03
MAX_GRAD_NORM = 10
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')


ImportError: 
AlbertConverter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment.


In [ ]:
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader


In [ ]:
# reference : https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index): 
        sentence = self.data.sentence[index].strip()
        word_labels = self.data.word_labels[index].split(",") 

        encoding = self.tokenizer(sentence,
                             
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        
        labels = [labels_to_ids[label] for label in word_labels] 
        encoded_labels = labels
        
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
            i += 1

        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(labels[0])
        
        return item

    def __len__(self):
        return self.len

In [ ]:
#reference : https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb

train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)

temp1 = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
val_size = .5
val_dataset = temp1

#test_dataset = temp1.drop(val_dataset.index).reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
#print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
validation_set = dataset(val_dataset, tokenizer, MAX_LEN)
#testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **test_params)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class clyas(nn.Module):
  
    def __init__(self,freeze_bert=False):
        super(clyas, self).__init__()
        self.auto =  AutoModel.from_pretrained('ai4bharat/indic-bert')
        self.classifier =  nn.Sequential(nn.Linear(768, 150), nn.ReLU(),nn.Linear(150, 80),nn.ReLU(),nn.Dropout(0.15),nn.Linear(80, 45),nn.ReLU(),nn.Dropout(0.15),nn.Linear(45,13))
        if freeze_bert:
            for param in self.auto.parameters():
                param.requires_grad = False
    def forward(self,ids,mask):
        output = self.auto(input_ids=ids,attention_mask=mask)
        hidden_stat = output[0][:, 0, :]
    
        logits = self.classifier(hidden_stat)
        return logits




In [ ]:
loss_fn = nn.CrossEntropyLoss()
bertt = clyas(True)
bertt.to(device)

In [ ]:
def train(epochs):

    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
  
    optimizer = torch.optim.Adam(params=bertt.parameters(), lr=LEARNING_RATE)
    bertt.train()
    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        logits = bertt(ids,mask)
        loss = loss_fn(logits, labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

        active_accuracy = labels.view(-1) != -100

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        torch.nn.utils.clip_grad_norm_(
            parameters=bertt.parameters(), max_norm=MAX_GRAD_NORM
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy= tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.485680341720581
Training loss per 100 training steps: 2.0068095267409145
Training loss per 100 training steps: 1.9640203231009679
Training loss per 100 training steps: 1.938724350889656
Training loss per 100 training steps: 1.9116133160127369
Training loss per 100 training steps: 1.8988819921801905
Training loss per 100 training steps: 1.8880609835245448
Training loss per 100 training steps: 1.8874738963796476
Training loss per 100 training steps: 1.8810114410337289
Training loss per 100 training steps: 1.8764276711578243
Training loss per 100 training steps: 1.8750850853029188
Training loss per 100 training steps: 1.8692599192843666
Training loss per 100 training steps: 1.8661638759355759
Training loss per 100 training steps: 1.8571280578665326
Training loss per 100 training steps: 1.8639614067700485
Training loss per 100 training steps: 1.8637437326442712
Training loss per 100 training steps: 1.8623453766982456
Training loss p

In [ ]:
def evaluate(bertt):
   
    # For each batch in our validation set...
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
  
    
    bertt.eval()
    with torch.no_grad():
       for idx, batch in enumerate(validation_loader):
             ids = batch['input_ids'].to(device, dtype = torch.long)
             mask = batch['attention_mask'].to(device, dtype = torch.long)
             labels = batch['labels'].to(device, dtype = torch.long)
    
             logits = bertt(ids,mask)
             loss = loss_fn(logits, labels)
             tr_loss += loss.item()
  
             nb_tr_steps += 1
             nb_tr_examples += labels.size(0)
    
             if idx % 100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Training loss per 100 training steps: {loss_step}")

             flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
             active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
             flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

             active_accuracy = labels.view(-1) != -100

             labels = torch.masked_select(flattened_targets, active_accuracy)
             predictions = torch.masked_select(flattened_predictions, active_accuracy)
          
             tr_labels.extend(labels)
             tr_preds.extend(predictions)
            
             tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
             tr_accuracy += tmp_eval_accuracy
  
    labels = [ids_to_labels[id.item()] for id in tr_labels]
    predictions = [ids_to_labels[id.item()] for id in tr_preds]
    
    eval_loss = tr_loss / nb_tr_steps 
    eval_accuracy = tr_accuracy / nb_tr_steps 
    print(f"Validation Loss: { eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy }")

    return labels, predictions

In [ ]:
x = evaluate(bertt)

Training loss per 100 training steps: 1.2055777311325073
Training loss per 100 training steps: 1.7776960647932374
Training loss per 100 training steps: 1.7565792283015464
Training loss per 100 training steps: 1.7646943089177838
Training loss per 100 training steps: 1.7672031529229182
Training loss per 100 training steps: 1.8002258670663167
Training loss per 100 training steps: 1.7914915562370255
Training loss per 100 training steps: 1.7799972818258316
Training loss per 100 training steps: 1.7733305906051404
Training loss per 100 training steps: 1.7788940567056029
Training loss per 100 training steps: 1.7626267935221012
Training loss per 100 training steps: 1.7626738626641862
Training loss per 100 training steps: 1.7715490925297153
Training loss per 100 training steps: 1.7783049409500367
Training loss per 100 training steps: 1.7810151228876645
Training loss per 100 training steps: 1.7895806548418323
Training loss per 100 training steps: 1.7966949432194792
Training loss per 100 training 

In [ ]:
pip install seqeval

     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=6c9dbc8e79fc8ef1517cdaefbeb81ebed732f4a406a4ff13ef0a90cb6fa406af
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
k = 0
for i in range(9064):
  if(x[0][i] ==  x[1][i]):
    k = k+1

In [ ]:
k

4767

TypeError: ignored

In [ ]:
PATH = "/content/drive/MyDrive/model.pth"

torch.save(bertt.state_dict(), PATH)

IsADirectoryError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


testing the model

In [ ]:
file_folder = r"hi_dev.conll"
file =  open(file_folder,encoding="utf-8")
text = file.read()


In [ ]:
import re

text = re.sub(r'[^\w\s]', '', text)


In [ ]:
s_cout = 0
l1 = []
l2 = []
l3 = []
for word in text.split('\n'):
  if(word == ''):
    s_cout =  s_cout + 1
  elif(word[0] == '#'):
    continue
    
  else:
      word = word.split(" ")
      k  = "sent_"+ str(s_cout)
      l1.append(k)
      l2.append(word[0])
      l3.append(word[-1])

In [ ]:
df.head()

,sentence,word_labels
0,१४९२,O
1,म,O
2,एक,O
3,चरटर,O
4,क,O


In [ ]:
df = pd.DataFrame({"sent":l1,"word":l2,"tag":l3})
df.head()

,sent,word,tag
0,sent_0,१४९२,O
1,sent_0,में,O
2,sent_0,एक,O
3,sent_0,चार्टर,O
4,sent_0,के,O


In [ ]:
lis = []
for i in range(df.shape[0]):
  if(df.iloc[i,1] != ''):
    lis.append(i)
df = df.iloc[lis,:]

In [ ]:
lis = []
for i in range(df.shape[0]):
  if(df.iloc[i,2] != ''):
    lis.append(i)
df = df.iloc[lis,:]

In [ ]:
df['sentence'] =  df['word']
df['word_labels']  = df['tag']
df.head()

,sent,word,tag,sentence,word_labels
0,sent_0,१४९२,O,१४९२,O
1,sent_0,में,O,में,O
2,sent_0,एक,O,एक,O
3,sent_0,चार्टर,O,चार्टर,O
4,sent_0,के,O,के,O


In [ ]:
labels_to_ids = {k: v for v, k in enumerate(df.tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(df.tag.unique())}
labels_to_ids
ids_to_labels

{0: 'O',
 1: 'B-LOC',
 2: 'B-PROD',
 3: 'B-CORP',
 4: 'I-CORP',
 5: 'B-GRP',
 6: 'I-GRP',
 7: 'B-CW',
 8: 'B-PER',
 9: 'I-PER',
 10: 'I-CW',
 11: 'I-LOC',
 12: 'I-PROD'}

In [ ]:
df = df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
df.head()

,sentence,word_labels
0,१४९२,O
1,में,O
2,एक,O
3,चार्टर,O
4,के,O


In [ ]:
df.shape

(3742, 2)

In [ ]:
testing_set = dataset(df, tokenizer, MAX_LEN)

In [ ]:
test_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
testing_set[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    2, 16900,  9573,  9758,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]),
 'labels': tensor(0),
 'offset_mapping': tensor([[0, 0],
         [0, 2],
         [2, 3],
         [3, 4],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0]]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
training_set[45]

{'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    2, 86184, 80134, 91123,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'labels': tensor(8),
 'offset_mapping': tensor([[ 0,  0],
         [ 0,  3],
         [ 3,  5],
         [ 5, 10],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
training_set[0]

NameError: ignored

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: ignored

In [ ]:
class clyas(nn.Module):
  
  def __init__(self,freeze_bert=False):
     super(clyas, self).__init__()
     self.auto =  AutoModel.from_pretrained('ai4bharat/indic-bert')
     self.classifier =  nn.Sequential(nn.Linear(768, 150), nn.ReLU(),nn.Linear(150, 80),nn.ReLU(),nn.Dropout(0.15),nn.Linear(80, 45),nn.ReLU(),nn.Dropout(0.15),nn.Linear(45,13))
     if freeze_bert:
            for param in self.auto.parameters():
                param.requires_grad = False
  def forward(self,ids,mask):
    output = self.auto(input_ids=ids,attention_mask=mask)
    hidden_stat = output[0][:, 0, :]
    
    logits = self.classifier(hidden_stat)
    return logits


In [ ]:
def evaluate(bertt):
   
    # For each batch in our validation set...
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
  
    
    bertt.eval()
    with torch.no_grad():
       for idx, batch in enumerate(testing_loader):
             ids = batch['input_ids'].to(device, dtype = torch.long)
             mask = batch['attention_mask'].to(device, dtype = torch.long)
             labels = batch['labels'].to(device, dtype = torch.long)
    
             logits = bertt(ids,mask)
             loss = loss_fn(logits, labels)
             tr_loss += loss.item()
  
             nb_tr_steps += 1
             nb_tr_examples += labels.size(0)
    
             if idx % 100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Training loss per 100 training steps: {loss_step}")

             flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
             active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
             flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

             active_accuracy = labels.view(-1) != -100

             labels = torch.masked_select(flattened_targets, active_accuracy)
             predictions = torch.masked_select(flattened_predictions, active_accuracy)
          
             tr_labels.extend(labels)
             tr_preds.extend(predictions)
            
             tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
             tr_accuracy += tmp_eval_accuracy
  
    labels = [ids_to_labels[id.item()] for id in tr_labels]
    predictions = [ids_to_labels[id.item()] for id in tr_preds]
    
    eval_loss = tr_loss / nb_tr_steps 
    eval_accuracy = tr_accuracy / nb_tr_steps 
    print(f"Validation Loss: { eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy }")

    return labels, predictions

In [ ]:
x = evaluate(bertt)

Training loss per 100 training steps: 1.558955430984497
Training loss per 100 training steps: 1.5182327781573381
Training loss per 100 training steps: 1.5105929386556445
Training loss per 100 training steps: 1.4917560229665814
Training loss per 100 training steps: 1.4872350966246646
Training loss per 100 training steps: 1.4714573921557672
Validation Loss: 1.4794737680823402
Validation Accuracy: 0.6734197218710493
